# Introduction

Well now that your long simulation is complete and your CSV file exported, let's get started with prepping this exported
CSV to be imported to create the lookup table!

The library importer code has been made to work with a very specifically formatted CSV file. If you follow the exact testbench setup guidelines there will be minimal changes.

If you made some crappy decisions like using a different setup notations than what were recommended let's play a game of Hide and seek. I hide and you seek professional help.

I have provided instructions to make import easier with minor differences in setup, but it won't be guaranteed to work!

Firstly, how should your exported CSV look? It should look exactly like shown [here](../../sample_files/MVE.csv). If it does not which could be the case for simulations run at corners other
than `Nominal` below are the instructions on how to format it.

1. If you see something which is not `Point,Test,Output,Nominal,Spec,Weight,Pass/Fail` or something similar at the first line delete anything before that.
2. If you also notice that the main result is in a different column and not directly beside the `Output` column and there could additionally, be `Min` and `Max` column. We will run the script below to rearrange the rows.
3. It is okay if your `Test` column does not match.
4. Go to the last line of your CSV file. Make sure there are no blank lines. The list line should be your last data line.
5. While eval errors are hard to spot in a large CSV file, they can sometimes be spotted at very end, with blank data columns instead of any data. You can copy a similar data point's data or rerun your simulation for that point. and enter the data. If you have multiple eval errors, resolve them and re-run simulations. **It is not recommended to remove the data points**

In [ ]:
import csv
# say if your main data for that corner resides in column 8
col_idx_of_interest = 8
with open('../../sample_files/data_out.csv', 'r') as f:
    data = list(csv.reader(f))
    print("Read the CSV file successfully")

with open ('../../sample_files/data_out_formatted.csv', 'w', newline='') as f:
    csv_out_formatted = csv.writer(f, delimiter=',')
    print("Created new CSV file for writing")
    row_idx = 0
    for row in data:
        if row[0].startswith('Parameters:'):
            # insert any changes you would like to make for Parameters row, specially like
            # making the keys compliant with the defaults of the library.
            # ds, gs, length, sb
        else:
            csv_out_formatted.writerow([row[0], row[1], row[2], row[col_idx_of_interest], row[3], row[4], row[5]]) # you can skip the extra columns if you want
        print(f"Processed Row: {row_idx}")
        row_idx += 1
print("New CSV file written successfully")

Since these demo jupyter notebooks are in different directory than the library and sometimes it might not be possible to install this library even if you have the dependencies, you can
run these demo files from within this directory by using the following snippet.

In [ ]:
import sys
import os

# Append the path of your library to the system path
sys.path.append(os.path.abspath("../../"))

Hopefully you now have a compliant file to be imported! However, one last step is to setup the `config`. We do so using the following snippet.
Make sure to provide the correct path appended with `conf.py` to the `write_config()` function. Everything depends on this file being at correct location.
So if you are running it from this directory then `../../` will take you to the root directory of this repository. From there `../../analog_daddy` takes you to the
correct folder. There you save a file `conf.py`. Hence the path is `../../analog_daddy/conf.py`.

(I agree there could be a more user friendly way of dealing with the above config file situation but I do not want to make any major changes to my code.)

What we essentially do with this program is to enter the PDK geometric limits, voltage limits as well as the keys in case you swayed away from the default naming conventions.
If not you can simply click enter if you want to keep the default values displayed. This function tries it's best to verify and sanitize the input.

After everything is done you should get a message `Configuration saved to conf.py!`. I encourage you to open this file in **Read-Only** mode and go through it for sanity check.

In [ ]:
from analog_daddy.write_config import write_config
write_config("../../analog_daddy/conf.py")

Phew! Now you are ready for import. Hopefully, there should not be any issues if you followed the instructions carefully.

In [ ]:
import numpy as np
from analog_daddy.importer import process_data, nest_populated_dictionary, lists_to_arrays
newx, newy = process_data('../../sample_files/GPDK45.csv')
z_ext = lists_to_arrays(nest_populated_dictionary(newy,newx))
# Make sure to add additional Information to the Array.
z_ext['nmos_svt']['w'] = 32e-06 # 32 um
z_ext['info'] = "GPDK 45 nm" # PDK Name
z_ext['temperature'] = 27 # 27 C
z_ext['corner'] = "tt" # Typical Typical
z_ext['pmos_svt']['w'] = 32e-06 # 32 um
# Once everything is done, save the arrays as numpy array and then save to npy file.
np.save("../../sample_files/GPDK45.npy",z_ext)
print("Done writing the npy file")

Now you are done! Head over to [Usage Demo](./usage_demo.ipynb)